# Week 9 (Aaron Kohn)
http://thinkstats2.com

Copyright 2016 Allen B. Downey

MIT License: https://opensource.org/licenses/MIT

## Exercise 11-1

In [1]:
import statsmodels.formula.api as smf
import nsfg
import first
import thinkstats2
import thinkplot
import numpy as np
import pandas as pd
import re

In [2]:

live, firsts, others = first.MakeFrames()
live = live[live.prglngth>30]
resp = nsfg.ReadFemResp()
resp.index = resp.caseid
join = live.join(resp, on='caseid', rsuffix='_r')

In [3]:
import patsy

def GoMining(df, dependent= 'prglngth'):
    """Searches for variables for prediction.

    df: DataFrame of pregnancy records

    returns: list of (rsquared, variable name) pairs
    """
    variables = []
    for name in df.columns:
        try:
            if df[name].var() < 1e-7:
                continue

            formula = dependent +' ~ ' + name
            
            # The following seems to be required in some environments
            # formula = formula.encode('ascii')

            model = smf.ols(formula, data=df)
            if model.nobs < len(df)/2:
                continue

            results = model.fit()
        except (ValueError, TypeError):
            continue

        variables.append((results.rsquared, name))

    return variables
def ReadVariables():
    """Reads Stata dictionary files for NSFG data.

    returns: DataFrame that maps variables names to descriptions
    """
    vars1 = thinkstats2.ReadStataDct('2002FemPreg.dct').variables
    vars2 = thinkstats2.ReadStataDct('2002FemResp.dct').variables

    all_vars = vars1.append(vars2)
    all_vars.index = all_vars.name
    return all_vars

def MiningReport(variables, n=30):
    """Prints variables with the highest R^2.

    t: list of (R^2, variable name) pairs
    n: number of pairs to print
    """
    all_vars = ReadVariables()

    variables.sort(reverse=True)
    for r2, name in variables[:n]:
        key = re.sub('_r$', '', name)
        try:
            desc = all_vars.loc[key].desc
            if isinstance(desc, pd.Series):
                desc = desc[0]
            print(name, r2, desc)
        except (KeyError, IndexError):
            print(name, r2)

In [4]:
variables = GoMining(join)
MiningReport(variables, n = 50)

prglngth 1.0 DURATION OF COMPLETED PREGNANCY IN WEEKS
wksgest 0.8062434116139242 GESTATIONAL LENGTH OF COMPLETED PREGNANCY (IN WEEKS)
totalwgt_lb 0.12445743148120214
birthwgt_lb 0.11977307804917103 BD-3 BIRTHWEIGHT IN POUNDS - 1ST BABY FROM THIS PREGNANCY
lbw1 0.10372542204583346 LOW BIRTHWEIGHT - BABY 1
mosgest 0.09562431989592668 GESTATIONAL LENGTH OF COMPLETED PREGNANCY (IN MONTHS)
prglngth_i 0.02205377579646839 PRGLNGTH IMPUTATION FLAG
canhaver 0.006050495268196232 DF-1 PHYSICALLY DIFFICULT FOR R TO HAVE A BABY
datcon01_i 0.005817755299879046 DATCON01 IMPUTATION FLAG
con1mar1_i 0.005546376136235764 CON1MAR1 IMPUTATION FLAG
nbrnaliv 0.004577565785532922 BC-2 NUMBER OF BABIES BORN ALIVE FROM THIS PREGNANCY
mar1con1_i 0.0031508022538595526 MAR1CON1 IMPUTATION FLAG
anynurse 0.0024520248837114345 BH-1 WHETHER R BREASTFED THIS CHILD AT ALL - 1ST FROM THIS PREG
bfeedwks 0.002369183944666786 DURATION OF BREASTFEEDING IN WEEKS
pregend1 0.002249389433799265 BC-1 HOW PREGNANCY ENDED - 1ST MEN

In [5]:
# model of variables that may be known to coworkers
model = smf.ols('prglngth ~ nbrnaliv>1 + paydu + diabetes', data=join)
results = model.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               prglngth   R-squared:                       0.013
Model:                            OLS   Adj. R-squared:                  0.012
Method:                 Least Squares   F-statistic:                     38.48
Date:                Tue, 11 May 2021   Prob (F-statistic):           1.07e-24
Time:                        19:41:21   Log-Likelihood:                -18241.
No. Observations:                8884   AIC:                         3.649e+04
Df Residuals:                    8880   BIC:                         3.652e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
========================================================================================
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
Intercept               38.7553      0.104    371.664      0.000      38.551      38.960
nbrnaliv > 1[T.True]    -1.5092      0.164     -9.188      0.000      -1.831      -1.187
paydu                   -0.1310      0.034     -3.856      0.000      -0.198      -0.064
diabetes                 0.0734      0.019      3.923      0.000       0.037       0.110
==============================================================================
Omnibus:                     1547.438   Durbin-Watson:                   1.619
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             6020.788
Skew:                          -0.829   Prob(JB):                         0.00
Kurtosis:                       6.676   Cond. No.                         42.2
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

## Exercise 11-3

In [6]:
model = smf.poisson('numbabes ~ age_r + C(race) + totincr + educat', data=join)
results = model.fit()
results.summary()


Optimization terminated successfully.
         Current function value: 1.687055
         Iterations 5


<class 'statsmodels.iolib.summary.Summary'>
"""
                          Poisson Regression Results                          
==============================================================================
Dep. Variable:               numbabes   No. Observations:                 8884
Model:                        Poisson   Df Residuals:                     8878
Method:                           MLE   Df Model:                            5
Date:                Tue, 11 May 2021   Pseudo R-squ.:                 0.03109
Time:                        19:41:21   Log-Likelihood:                -14988.
converged:                       True   LL-Null:                       -15469.
Covariance Type:            nonrobust   LLR p-value:                1.106e-205
================================================================================
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept        1.0842      0.045     23.995      0.000       0.996       1.173
C(race)[T.2]    -0.1398      0.015     -9.464      0.000      -0.169      -0.111
C(race)[T.3]    -0.0914      0.025     -3.717      0.000      -0.140      -0.043
age_r            0.0208      0.001     20.474      0.000       0.019       0.023
totincr         -0.0179      0.002     -9.442      0.000      -0.022      -0.014
educat          -0.0443      0.003    -15.139      0.000      -0.050      -0.039
================================================================================
"""

In [7]:
columns = ['totincr', 'age_r', 'educat', 'race']
new = pd.DataFrame([[14, 35, 16, 1]], columns=columns)
y = results.predict(new)
y

0    2.342182
dtype: float64

The average 35 year old black woman makeing over $75,000 would be predicted to have 2 children. 

## Exercise 11-4

In [8]:
model = smf.mnlogit('rmarital ~ age_r + C(race) + totincr + educat', data=join)
results = model.fit()
results.summary()


Optimization terminated successfully.
         Current function value: 1.087603
         Iterations 8


<class 'statsmodels.iolib.summary.Summary'>
"""
                          MNLogit Regression Results                          
==============================================================================
Dep. Variable:               rmarital   No. Observations:                 8884
Model:                        MNLogit   Df Residuals:                     8854
Method:                           MLE   Df Model:                           25
Date:                Tue, 11 May 2021   Pseudo R-squ.:                  0.1655
Time:                        19:41:24   Log-Likelihood:                -9662.3
converged:                       True   LL-Null:                       -11579.
Covariance Type:            nonrobust   LLR p-value:                     0.000
================================================================================
  rmarital=2       coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept        4.4532      0.279     15.977      0.000       3.907       5.000
C(race)[T.2]    -0.9219      0.089    -10.409      0.000      -1.095      -0.748
C(race)[T.3]    -0.6334      0.136     -4.674      0.000      -0.899      -0.368
age_r           -0.0570      0.006     -9.754      0.000      -0.068      -0.046
totincr         -0.1302      0.012    -11.298      0.000      -0.153      -0.108
educat          -0.2051      0.019    -11.017      0.000      -0.242      -0.169
--------------------------------------------------------------------------------
  rmarital=3       coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept       -4.5432      0.916     -4.960      0.000      -6.338      -2.748
C(race)[T.2]    -0.4405      0.236     -1.865      0.062      -0.904       0.023
C(race)[T.3]     0.0329      0.335      0.098      0.922      -0.623       0.689
age_r            0.1301      0.019      6.833      0.000       0.093       0.167
totincr         -0.3251      0.032    -10.182      0.000      -0.388      -0.263
educat          -0.1010      0.048     -2.100      0.036      -0.195      -0.007
--------------------------------------------------------------------------------
  rmarital=4       coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept       -2.6485      0.302     -8.783      0.000      -3.239      -2.057
C(race)[T.2]    -0.3188      0.093     -3.417      0.001      -0.502      -0.136
C(race)[T.3]    -0.7664      0.171     -4.485      0.000      -1.101      -0.431
age_r            0.0641      0.007      9.817      0.000       0.051       0.077
totincr         -0.2270      0.012    -19.588      0.000      -0.250      -0.204
educat           0.0673      0.017      4.036      0.000       0.035       0.100
--------------------------------------------------------------------------------
  rmarital=5       coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept        0.8231      0.345      2.385      0.017       0.147       1.500
C(race)[T.2]    -1.0491      0.104    -10.125      0.000      -1.252      -0.846
C(race)[T.3]    -0.5644      0.156     -3.624      0.000      -0.870      -0.259
age_r            0.0063      0.007      0.872      0.383      -0.008       0.020
totincr         -0.2923      0.015    -20.149      0.000      -0.321      -0.264
educat          -0.0136      0.021     -0.639      0.523      -0.056       0.028
--------------------------------------------------------------------------------
  rmarital=6       coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept        6.2370      0.275     22.654      0.000       5.697       6.777
C

In [9]:
columns = ['totincr', 'age_r', 'educat', 'race']
new = pd.DataFrame([[11, 25, 12, 2]], columns=columns)
y = results.predict(new)
y = y.rename(columns = {0 : 'married', 1 : 'cohabitating', 2 : 'widowed', 3 : 'divorced', 4 : 'seperated', 5 : 'never married'})
y

,married,cohabitating,widowed,divorced,seperated,never married
0,0.748384,0.125474,0.001103,0.035295,0.023813,0.065931
